In [2]:
%matplotlib inline
from random import randint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from sklearn.model_selection import train_test_split
import nltk
import gensim
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.layers.embeddings import Embedding

Using TensorFlow backend.
D:\Anaconda\envs\tensorflow\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [91]:
dataset = pd.read_csv("D:\\PESU\\Sem_V\\AI\\_Lab_\\Lab2\\consolidated.csv")

In [92]:
dataset.head()

,Unnamed: 0,tweet,corrected,sentiment,valid
0,0,xxxSNAMEyyy communication was sent in the mont...,xxxSNAMEyyy communication was sent in the mont...,1,1
1,1,xxxSNAMEyyy xxxSNAMEyyy xxxSNAMEyyy xxxSNAMEyy...,xxxSNAMEyyy xxxSNAMEyyy xxxSNAMEyyy xxxSNAMEyy...,1,1
2,2,xxxSNAMEyyy How do I find out why my credit ca...,xxxSNAMEyyy How do I find out why my credit ca...,1,1
3,3,xxxSNAMEyyy Refund issues ..DM u the details.....,xxxSNAMEyyy Refund issues . Direct Message you...,1,1
4,4,xxxSNAMEyyy New s/w upgrade xxxSNAMEyyy is deb...,xxxSNAMEyyy New software upgrade xxxSNAMEyyy i...,0,1


In [93]:
dataset = dataset[dataset["valid"] == 1]
dataset.head()

,Unnamed: 0,tweet,corrected,sentiment,valid
0,0,xxxSNAMEyyy communication was sent in the mont...,xxxSNAMEyyy communication was sent in the mont...,1,1
1,1,xxxSNAMEyyy xxxSNAMEyyy xxxSNAMEyyy xxxSNAMEyy...,xxxSNAMEyyy xxxSNAMEyyy xxxSNAMEyyy xxxSNAMEyy...,1,1
2,2,xxxSNAMEyyy How do I find out why my credit ca...,xxxSNAMEyyy How do I find out why my credit ca...,1,1
3,3,xxxSNAMEyyy Refund issues ..DM u the details.....,xxxSNAMEyyy Refund issues . Direct Message you...,1,1
4,4,xxxSNAMEyyy New s/w upgrade xxxSNAMEyyy is deb...,xxxSNAMEyyy New software upgrade xxxSNAMEyyy i...,0,1


In [94]:
all_tweets = list(map(str, list(dataset['tweet'])))

In [95]:
len(all_tweets)

16763

In [96]:
dataset = dataset.sample(n=5500)

In [97]:
X = dataset["tweet"]
Y = dataset["corrected"]

In [98]:
seed = 123
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.0909, random_state=seed)

In [99]:
len(X_train)

5000

In [100]:
X_train = list(map(str, list(X_train)))
X_test = list(map(str, list(X_test)))

In [101]:
all_tweets_tokenized = [nltk.word_tokenize(x.lower()) for x in all_tweets]
tokenized_tweets = [nltk.word_tokenize(x.lower()) for x in X_train]

In [103]:
len(all_tweets_tokenized)

16763

In [104]:
max_length_tweet = len(max(all_tweets_tokenized, key = lambda x: len(x))) + 3

In [105]:
max_length_tweet

49

In [107]:
for i in range(len(tokenized_tweets)):
    while(len(tokenized_tweets[i]) < max_length_tweet):
        tokenized_tweets[i] += [""]
for i in range(len(all_tweets_tokenized)):
    while(len(all_tweets_tokenized[i]) < max_length_tweet):
        all_tweets_tokenized[i] += [""]

In [110]:
len(all_tweets_tokenized[15])

49

In [112]:
gensim_model = gensim.models.Word2Vec(all_tweets_tokenized, min_count = 1, size = 100, workers = 4, sample = 1e-3)

In [113]:
print(gensim_model)

Word2Vec(vocab=13191, size=100, alpha=0.025)


In [114]:
X_train_new = []
for i in tokenized_tweets:
    x = []
    for j in i:
        x.append(list(gensim_model[j]))
    X_train_new.append(x)

In [117]:
len(X_train_new[1])

49

In [118]:
len(X_train_new[0][0])

100

In [119]:
tokenizer = Tokenizer()

In [120]:
Y = list(map(str, list(Y)))

In [121]:
tokenizer.fit_on_texts(Y)
words = tokenizer.word_counts

In [122]:
import operator
sorted_x = sorted(words.items(), key=operator.itemgetter(1))

In [123]:
words['unknown'] = 200000
len(words)

5560

In [124]:
final=sorted_x[len(words)-3501:len(words)]

In [126]:
len(final)
final[-1]

('xxxsnameyyy', 7258)

In [127]:
listi=[] # Most occurring 3501 words
for i in final[::-1]:
    listi.append(i[0])

In [129]:
(listi[:5])

['xxxsnameyyy', 'to', 'the', 'you', 'your']

In [130]:
values = array(listi)
#print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
#print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [131]:
d = {}
xyz = 0
for i in listi:
    d[i] = integer_encoded[xyz][0]
    xyz += 1

In [132]:
y_train = list(map(str, list(y_train)))
y_train[:5]

['xxxSNAMEyyy relates to failed POS transaction , please share the details at xxxURLyyy under ATM-related or POS ',
 'xxxSNAMEyyy xxxHASHTAGyyy A2 ) . B ) . Dusshera',
 'xxxSNAMEyyy If you want to close your PayPal account you still need to reach out to xxxSNAMEyyy here . Thanks',
 'xxxSNAMEyyy I can nor for the life of me get my account to recognise my new address as my primary. Please help!',
 'xxxSNAMEyyy Just look at how private banks treat their customers. Whether its providing some information or service. S xxxURLyyy']

In [133]:
tokenized_tweets_correct = [nltk.word_tokenize(x.lower()) for x in y_train]

In [134]:
max_length_tweet_correct = max_length_tweet

In [135]:
max_length_tweet_correct

49

In [136]:
for i in range(len(tokenized_tweets_correct)):
    while(len(tokenized_tweets_correct[i]) < max_length_tweet_correct):
        tokenized_tweets_correct[i] += [""]

In [137]:
len(min(tokenized_tweets_correct, key = lambda x: len(x)))

49

In [138]:
y_train_new = []
for i in tokenized_tweets_correct:
    x = []
    for j in i:
        try:
            x.append(onehot_encoded[listi.index(j)])
        except:
            x.append(onehot_encoded[listi.index('unknown')])
    y_train_new.append(x)

In [227]:
listi.index('unknown')

1360

In [139]:
list(onehot_encoded[listi.index('unknown')]).index(1)

3257

In [140]:
len(y_train_new[0][0])

3501

In [141]:
# configure problem
n_inp_features = 100
n_timesteps_in = max_length_tweet
n_timesteps_out = max_length_tweet_correct
n_op_features = 3501

In [142]:
n_timesteps_in

49

In [143]:
model = Sequential()
model.add(LSTM(150, input_shape=(n_timesteps_in, n_inp_features)))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(150, return_sequences=True))
model.add(TimeDistributed(Dense(n_op_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [172]:
(X_train_new[0])

[[-0.30236256,
  0.66680604,
  0.94770563,
  0.82024562,
  1.2294565,
  -0.7331273,
  -0.50354826,
  0.58084637,
  0.50071889,
  0.51027387,
  1.3574052,
  0.97152221,
  -0.77275914,
  -0.42760578,
  -0.10996114,
  -1.1243081,
  0.25662595,
  -0.70922977,
  -0.36731413,
  0.77511781,
  0.48756069,
  0.36089116,
  -0.98500478,
  0.77959412,
  0.08664231,
  -0.19290231,
  -0.038805641,
  -1.6626281,
  -0.1256668,
  0.10657816,
  0.15762091,
  -1.2531041,
  -0.15050623,
  -0.068452515,
  -0.38438937,
  -0.25181821,
  1.1293417,
  1.3554165,
  0.67014819,
  1.1645613,
  0.40255666,
  0.047926147,
  -1.6094158,
  -0.32690582,
  -0.36953139,
  0.079189226,
  -0.87365586,
  0.50317872,
  -1.0109855,
  -0.8545692,
  -0.47191399,
  0.051834051,
  -0.88357359,
  -0.39144182,
  -0.54458779,
  -1.1414546,
  -0.63062495,
  -1.3263495,
  -0.52940029,
  0.25577947,
  -1.3468035,
  -0.28291258,
  0.53797925,
  0.043762248,
  -0.033759501,
  0.052376989,
  -0.026377413,
  -1.5247343,
  0.37633669,
  -0

In [262]:
model.fit(X_train_new, y_train_new, validation_split = 0.20, epochs=2)

Train on 4000 samples, validate on 1000 samples
Epoch 1/2
4000/4000 [==============================] - 295s - loss: 2.1595 - acc: 0.6870 - val_loss: 2.1899 - val_acc: 0.6838
Epoch 2/2
4000/4000 [==============================] - 127s - loss: 2.1526 - acc: 0.6874 - val_loss: 2.1853 - val_acc: 0.6838


In [263]:
# serialize model to JSON
model_json = model.to_json()
with open("D:\\PESU\\Sem_V\\AI\\_Lab_\\Lab2\\model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("D:\\PESU\\Sem_V\\AI\\_Lab_\\Lab2\\model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('D:\PESU\Sem_V\AI\_Lab_\Lab2\model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights('D:\PESU\Sem_V\AI\_Lab_\Lab2\model.h5')
print("Loaded model from disk")

In [264]:
tokenized_test_tweets = [nltk.word_tokenize(x.lower()) for x in X_test]

In [266]:
tokenized_test_tweets[:2]

[['xxxsnameyyy',
  'xxxsnameyyy',
  'there',
  'are',
  'two',
  'cash',
  'mashin',
  'sector',
  '14',
  'gurugram',
  'huda',
  'market',
  'branch',
  'but',
  'both',
  'was',
  'not',
  'working',
  'last',
  '1',
  'months',
  '.'],
 ['xxxsnameyyy',
  'our',
  'phone',
  'banking',
  'numbers',
  ':',
  '1-860-419-5555',
  'and',
  '1-860-500-5555',
  'to',
  'inquire',
  'about',
  'the',
  'status',
  '.',
  'please',
  'write',
  'back',
  'to',
  'us',
  'in',
  '(',
  '2/3']]

In [231]:
max_length_test = max([len(x) for x in tokenized_test_tweets])
max_length_test = max_length_tweet
max_length_test

48

In [267]:
for i in range(len(tokenized_test_tweets)):
    while(len(tokenized_test_tweets[i]) < max_length_tweet):
        tokenized_test_tweets[i] += [""]

In [233]:
#gensim_model = gensim.models.Word2Vec(tokenized_test_tweets, min_count = 1, size = 100, workers = 4, sample = 1e-3)

In [268]:
print(gensim_model)

Word2Vec(vocab=13191, size=100, alpha=0.025)


In [269]:
X_test_new = []
for i in tokenized_test_tweets:
    x = []
    for j in i:
        x.append(list(gensim_model[j]))
    X_test_new.append(x)

In [270]:
min([len(x) for x in X_test_new]) == max([len(x) for x in X_test_new])

True

In [272]:
len(X_test_new[0])

49

In [275]:
y_test = list(y_test)
y_test[:2]

['xxxSNAMEyyy xxxSNAMEyyy there are two cash machine in sector 14 gurugram huda market branch but both was not working last 1 months .',
 'xxxSNAMEyyy our Phone Banking numbers : 1-860-419-5555 and 1-860-500-5555 to inquire about the status . Please write back to us in ( 2/3']

In [276]:
y_test = list(map(str, y_test))

In [277]:
y_test[0:2]

['xxxSNAMEyyy xxxSNAMEyyy there are two cash machine in sector 14 gurugram huda market branch but both was not working last 1 months .',
 'xxxSNAMEyyy our Phone Banking numbers : 1-860-419-5555 and 1-860-500-5555 to inquire about the status . Please write back to us in ( 2/3']

In [278]:
tokenized_tweets_correct_test = [nltk.word_tokenize(x.lower()) for x in y_test]
tokenized_tweets_correct_test

[['xxxsnameyyy',
  'xxxsnameyyy',
  'there',
  'are',
  'two',
  'cash',
  'machine',
  'in',
  'sector',
  '14',
  'gurugram',
  'huda',
  'market',
  'branch',
  'but',
  'both',
  'was',
  'not',
  'working',
  'last',
  '1',
  'months',
  '.'],
 ['xxxsnameyyy',
  'our',
  'phone',
  'banking',
  'numbers',
  ':',
  '1-860-419-5555',
  'and',
  '1-860-500-5555',
  'to',
  'inquire',
  'about',
  'the',
  'status',
  '.',
  'please',
  'write',
  'back',
  'to',
  'us',
  'in',
  '(',
  '2/3'],
 ['xxxsnameyyy', 'with', 'your', 'personal', 'details', 'displayed', '.'],
 ['xxxsnameyyy',
  'hi',
  'jonathan',
  '.',
  'please',
  'allow',
  'us',
  'some',
  'time',
  'to',
  'research',
  'your',
  'account',
  'and',
  'we',
  'will',
  'connect',
  'with',
  'you',
  '.'],
 ['xxxsnameyyy',
  'hello',
  '!',
  'we',
  'are',
  'sorry',
  'about',
  'the',
  'trouble',
  'reaching',
  'our',
  'phone',
  'teammates',
  '.',
  'for',
  'assistance',
  ',',
  'please',
  'send',
  'us',


In [279]:
max_length_tweet

49

In [280]:
for i in range(len(tokenized_tweets_correct_test)):
    while(len(tokenized_tweets_correct_test[i]) < max_length_tweet):
        tokenized_tweets_correct_test[i] += [""]

In [281]:
len(max(tokenized_tweets_correct_test, key = lambda x: len(x)))

49

In [282]:
y_test_new = []
for i in tokenized_tweets_correct_test:
    x = []
    for j in i:
        try:
            x.append(onehot_encoded[listi.index(j)])
        except:
            x.append(onehot_encoded[listi.index('unknown')])
    y_test_new.append(x)
   

In [284]:
y_test_new[:1]

[[array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0., 

In [286]:
y_train_new

[[array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
  array([ 0.,  0.,  0., ...,  0., 

In [285]:
max([len(x) for x in y_test_new])

49

In [287]:
scores = model.evaluate(X_test_new,y_test_new)
print("Accuracy: %.2f%%" % (scores[1]*100))

500/500 [==============================] - 4s     
Accuracy: 68.61%
